In [25]:
import os
import requests
import pandas as pd

In [87]:
from dotenv import load_dotenv
load_dotenv()
apiKey = os.getenv("keygoogle")


In [33]:
#Creamos una funcion para hacer un request a google.
def getData(query, page_token=None):
    url="https://maps.googleapis.com/maps/api/place/textsearch/json?"
    params= {"key":os.getenv("keygoogle"), "query":query}
    if page_token:
        params["pageToken"]=page_token
    res= requests.get(url, params=params)
    res= res.json()
    results=res["results"]
    if "next_page_token" in res.keys():
        page_token = res["next_page_token"]
        results=[results, page_token]
    else:
        results=results,False
    return results

In [34]:
ok=os.getenv("keygoogle")

In [46]:
#Pedimos los starbucks cerca de Mountain View, California.
data_starbucks,next_page= getData("starbucks in Mountain View, California ")

In [52]:
#Nos da un total de 20, se que hay más y que te pueden dar otra "pagina", pero no he tenido tiempo para saber como se hace...
data = pd.DataFrame(data_starbucks)
len(data)

20

In [65]:
#A continuación creamos un bucle para hacer de la request un dataframe. 
Starbucks = []
for coffe in data["geometry"]:
    new_dict={}
    new_dict["Latitude"]= coffe['location']['lat']
    new_dict["Longitude"]= coffe['location']['lng']
    Starbucks.append(new_dict)
Starbucks = pd.DataFrame(Starbucks)

In [66]:
Starbucks

,Latitude,Longitude
0,37.387571,-122.083068
1,37.378791,-122.070671
2,37.402755,-122.112379
3,37.390873,-122.094709
4,37.415785,-122.077546
5,37.411440,-122.094239
6,37.421800,-122.096419
7,37.403017,-122.079089
8,37.372255,-122.088235
9,37.402120,-122.111866


In [71]:

def transformToGeoPoint(s):
    return {
        "type":"Point",
        "coordinates":[s.Longitude, s.Latitude]
    }

In [72]:
#Convertimos la latitud y longitud en Geopoint.
Starbucks["geopoint"] = Starbucks.apply(transformToGeoPoint, axis=1)

In [73]:
Starbucks

,Latitude,Longitude,geopoint
0,37.387571,-122.083068,"{'type': 'Point', 'coordinates': [-122.083068,..."
1,37.378791,-122.070671,"{'type': 'Point', 'coordinates': [-122.070671,..."
2,37.402755,-122.112379,"{'type': 'Point', 'coordinates': [-122.112379,..."
3,37.390873,-122.094709,"{'type': 'Point', 'coordinates': [-122.0947095..."
4,37.415785,-122.077546,"{'type': 'Point', 'coordinates': [-122.077546,..."
5,37.411440,-122.094239,"{'type': 'Point', 'coordinates': [-122.094239,..."
6,37.421800,-122.096419,"{'type': 'Point', 'coordinates': [-122.096419,..."
7,37.403017,-122.079089,"{'type': 'Point', 'coordinates': [-122.079089,..."
8,37.372255,-122.088235,"{'type': 'Point', 'coordinates': [-122.088235,..."
9,37.402120,-122.111866,"{'type': 'Point', 'coordinates': [-122.111866,..."


In [74]:
#Lo exportamos en Json para subirlo a mongo, con la intencion de realizar la query near, que no he podido...
Starbucks.to_json("../input/Starbucks.json",orient="records")